### Classification des avis cinématographiques


In [1]:
# Bibliothèques nécessaires
import random
import time
import pandas as pd # pandas pour avoir un format (DataFrame) confortable pour les données.
import numpy as np # Numpy pour le calcul du taux de bonnes prédictions
from sklearn.feature_extraction.text import TfidfVectorizer # outil pour traiter le texte
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics

#### Création du fichier pour la validation

In [2]:
# On charge le fichier où sont contenues les 25000 données
train = pd.read_csv("data/train_data_original/labeledTrainData.tsv", header=0,delimiter="\t", quoting=3)

print train["review"].size
# On sélectionne 5000 id au hasard parmis les 25000
movie_id = random.sample(range(train["review"].size), 5000)

# On enregistre le fichier avec les revues pour le test
test_validation = [{"id":train["id"][data], "review":train["review"][data]} for data in movie_id]
test_validation = pd.DataFrame(test_validation)
test_validation.to_csv("data/validation_data/test_validation.csv",index=False)

# On enregistre les sentiments des 5000 pour la validation
validation = [{"id":train["id"][data], "sentiment":train["sentiment"][data]} for data in movie_id]
validation = pd.DataFrame(validation)
validation.to_csv( "data/validation_data/validation.csv",index=False)

# On enregistre le fichier d'apprentissage en enlevant les 5000 données.
training_set = [{"id":train["id"][data], "sentiment":train["sentiment"][data],"review":train["review"][data]}
                for data in range(train['review'].size) if not data in movie_id]

training_set = pd.DataFrame(training_set)
training_set.to_csv("data/train_data/labeledTrainData.csv",index=False)

25000


#### Avec les RF

In [3]:
# Définition de la pipeline
rf_clf = Pipeline([('vect', TfidfVectorizer(analyzer = "word",stop_words='english',max_features = 5000)),
                   ('clf', RandomForestClassifier(n_estimators = 150))])

# Chargement du fichier contenant les données d'apprentissage
train = pd.read_csv("data/train_data/labeledTrainData.csv", header=0,delimiter=",")

# Apprentissage
tps1 = time.clock()
rf_clf = rf_clf.fit(train["review"],train["sentiment"])
tps2 = time.clock() 
print "Training done in ",tps2 - tps1," seconds."
print""

# Données à analyser
test = pd.read_csv("data/validation_data/test_validation.csv", header=0, delimiter=",")

# Prédictions
tps1 = time.clock()
result = rf_clf.predict(test["review"])
tps2 = time.clock() 
print "Predictions done in ",tps2 - tps1," seconds."
print""

# On sauvegarde le resultat
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} ).to_csv( "data/validation_data//resultats/resultat_RF.csv", index=False, quoting=3)

# On calcul le taux de bonnes prédictions
valid = pd.read_csv("data/validation_data/validation.csv", header=0, delimiter=",") # Fichier contenant les sentiments valides
print 'Match rate: ',np.mean(result == valid["sentiment"])*100,"%"
print''
print 'Confusion Matrix:'
metrics.confusion_matrix(valid["sentiment"],result)

Training done in  74.5862684116  seconds.

Predictions done in  1.6419245717  seconds.

Match rate:  84.74 %

Confusion Matrix:


array([[2105,  342],
       [ 421, 2132]])

#### Analyse du fichier de 50000 lignes



In [4]:
# On charge le fichier à analyser
test = pd.read_csv("data/test_data/unlabeledTrainData.tsv", header=0, delimiter="\t",quoting=3 )

# Prédictions
tps1 = time.clock() 
res_rf = rf_clf.predict(test["review"]) # avec les forêts aléatoires
tps2 = time.clock() 
print "Predictions done in ",tps2 - tps1," seconds."
print""

# On enregistre les résultats
output_rf = pd.DataFrame( data={"id":test["id"], "sentiment":res_rf} ).to_csv( "data/test_data/resultats/predic_rf.csv", index=False, quoting=3 )

Predictions done in  16.5828477298  seconds.



#### Avec une SVM un peu plus péchu

In [5]:
# Définition de la pipeline
svmPlus_clf = Pipeline([('vect', TfidfVectorizer(analyzer = "word",stop_words='english',max_features = 5000)),
                        ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-4, n_iter=50, random_state=42))])

# Chargement du fichier contenant les données d'apprentissage
train = pd.read_csv("data/train_data/labeledTrainData.csv", header=0,delimiter=",")

# Apprentissage
tps1 = time.clock()
svmPlus_clf = svmPlus_clf.fit(train["review"],train["sentiment"])
tps2 = time.clock()
print "Training done in ",tps2 - tps1," seconds."
print""

# Données à analyser
test = pd.read_csv("data/validation_data/test_validation.csv", header=0, delimiter=",")
    
# Prédictions 
tps1 = time.clock()
result = svmPlus_clf.predict(test["review"])
tps2 = time.clock() 
print "Predictions done in ",tps2 - tps1," seconds."
print""

# On sauvegarde le resultat
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} ).to_csv( "data/validation_data/resultats/resultat_SVMPlus.csv", index=False, quoting=3)

# On calcul le taux de bonnes prédictions
valid = pd.read_csv("data/validation_data/validation.csv", header=0, delimiter=",") # Fichier contenant les sentiments valides
print 'Match rate: ',np.mean(result == valid["sentiment"])*100,"%"
print''
print 'Confusion Matrix:'
metrics.confusion_matrix(valid["sentiment"],result)

Training done in  5.35520738599  seconds.

Predictions done in  1.17070146651  seconds.

Match rate:  88.1 %

Confusion Matrix:


array([[2155,  306],
       [ 289, 2250]])

#### Analyse du fichier de 50000 lignes avec la SVM péchu

In [6]:
# On charge le fichier à analyser
test = pd.read_csv("data/test_data/unlabeledTrainData.tsv", header=0, delimiter="\t",quoting=3 )

# Prédictions
tps1 = time.clock() 
res_svmPlus = svmPlus_clf.predict(test["review"]) # avec les SVM améliorée
tps2 = time.clock() 
print 'Done in',tps2 - tps1,'seconds.'
print ""

# On enregistre les résultats
output_SVM = pd.DataFrame( data={"id":test["id"], "sentiment":res_svmPlus} ).to_csv( "data/test_data/resultats/predic_svmPlus.csv", index=False, quoting=3 )

# Match rate
out_svm = pd.read_csv("data/test_data/resultats/predic_svmPlus.csv", header=0, delimiter=",")
out_rf = pd.read_csv("data/test_data/resultats/predic_rf.csv", header=0, delimiter=",")

match_rate = 100.*sum([1. for el in range(out_svm["sentiment"].size) if out_svm["sentiment"][el] == out_rf["sentiment"][el]])/out_svm["sentiment"].size

print "Match rate between RF and SVM: ",match_rate,"%"

Done in 11.5040630298 seconds.

Match rate between RF and SVM:  87.87 %


In [ ]:
from time import time

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

from sklearn import manifold
from sklearn.utils import check_random_state

t0 = time()
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
trans_data = tsne.fit_transform(sphere_data).T
t1 = time()
print("t-SNE: %.2g sec" % (t1 - t0))

TfidfVectorizer(analyzer = "word",stop_words='english',max_features = 5000)